# 1. Self-Attention

### 1.1 Seq2Seq

<img src='attachment/43. Sequence.jpg' style='zoom:50%'/>

1. 传统的基于RNN的Seq2Seq，主要有两个缺点: 
    - <font color=blue>不方便并行计算</font>;
    - Encoder学得的固定长度context vector无法涵盖较长的句子。


2. CNN也能实现Seq2Seq，同时可以并行计算，但是低层的Filter只能考虑有限的上下文，若要考虑全局的Context，需要叠加多层Filter。

### 1.2 Self-Attention

<table>
    <tr>
        <td><img src='attachment/43. Self-Attention_1.jpg' style='zoom:50%'></td>
        <td><img src='attachment/43. Self-Attention_2.jpg' style='zoom:50%'></td>
    </tr>
    <tr>
        <td><img src='attachment/43. Self-Attention_3.jpg' style='zoom:50%'></td>
        <td><img src='attachment/43. Self-Attention_4.jpg' style='zoom:50%'></td>
    </tr>
</table>

#### Step1

$$
a^{i} = Wx^{i}
$$

#### Step2
$q:$ query(to match others)  

$k:$ key(to be matched)  

$v:$ information to be extracted  

$$
q^{i} = W^{q}a^{i} \\
k^{i} = W^{k}a^{i} \\
v^{i} = W^{v}a^{i} \\
$$

#### Step3: 计算Attention

$a_{ij}:~$ 第$~i~$个位置相对第$~j~$个位置的注意力Attention。

$$
\begin{align}
a_{ij} &= \frac{q^{i} \cdot k^{j}}{\sqrt{d}} \\
\hat{a_{ij}} &= \frac{ a_{ij} }{ \sum_{j}a_{ij} }
\end{align}
$$

$d: ~$ $k$的dimension，是为了梯度的稳定。

#### Step4:  对各位置的information加权平均

$$
\begin{align}
b_{i } = \sum_{j} \hat{a}_{ij} v^{j}
\end{align}
$$

<font color=red>可以看到，Self-Attention完全摒弃了CNN，RNN思想。</font>

### 1.3 Selft-Attention的并行计算

<table>
    <tr>
        <td><img src='attachment/43. Self-Attention_parallel_1.jpg' style='zoom:50%'></td>
        <td><img src='attachment/43. Self-Attention_parallel_2.jpg' style='zoom:50%'></td>
    </tr>
    <tr>
        <td><img src='attachment/43. Self-Attention_parallel_3.jpg' style='zoom:50%'></td>
        <td><img src='attachment/43. Self-Attention_parallel_4.jpg' style='zoom:50%'></td>
    </tr>
</table>

### 1.4 Multi-head Self-Attention

<img src='attachment/43. Multi-head Self-Attention.png' style='zoom:50%'>

### 1.5 Positional Encoding

截止目前为止，我们介绍的Transformer模型并没有捕捉顺序序列的能力，也就是说无论句子的结构怎么打乱，Transformer都会得到类似的结果。

为了解决这个问题，作者引入了位置编码（Position Embedding）特征，具体地说，位置编码会在词向量中加入单词的位置信息，这样Transformer就能区分不同位置的单词。

常用的模式有：
- 根据数据学习；

- 自己设计编码规则


$$
\begin{align}
PE_{(pos, 2i)} &= \sin \left(pos/10000^{2i/d_{k}} \right) \\
PE_{(pos, 2i+1)} &= \cos \left(pos/10000^{2i/d_{k}} \right) 
\end{align}
$$

式中，$pos~$表示位置索引，$~i~$表示词向量的维度索引，$d_{k}$表示词向量长度。

<img src='attachment/43. Positional Encoding.jpg' style='zoom:50%'>

### 1.6 Seq2Seq with Attention

<img src='attachment/43. Seq2Seq based on Self-Attention.png' style='zoom:50%'>

# 2. Transformer

Transformer: Seq2Seq model with "Self-Attention".

<img src='attachment/43. Transformer.jpg' style='zoom:40%'>

### 2.1 Transformer Block

<img src='attachment/43. Transformer block.png' style='zoom:40%'>

1. 蓝色的线是残差连接（residual connections），可以用来防止梯度消失或梯度爆炸；


2. Layer Normalization可以使训练更快:

$$
\hat{\pmb{a}} = \gamma \frac{\pmb{a} - \mu}{\sqrt{\sigma^{2} + \epsilon}} + \beta
$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\epsilon~$用来防止分母为0，$\gamma, \beta$是可学习的参数。

3. Feed Forward层对每个输入vector，都单独使用个MLP（Multi-layer Perceptron）进行前向计算。

#### layer norm vs. batch norm

<img src='attachment/43. Layer-Norm vs Batch Norm.jpg' style='zoom:50%'>

### 2.2 Application

#### 1. Sentiment Classification

<img src='attachment/43. Sentiment Classification.jpg' style='zoom:50%'>

#### 2. Auto-regression Generation

比如，输入"hell"，预测下一个字符为"o"，生成"hello". 


<img src='attachment/43. Auto-regression Generation.jpg' style='zoom:50%'>

在训练上面的网络时，会对输出的每一个character建立loss。但是，<font color=blue>self-attention机制本身可以看到前面的character</font>。因此我们在训练时有必要不让transformer不看到前面的character。

<img src='attachment/43. mask attention.png' style='zoom:50%'>

### 2.3 Full Architecture of Transformer

<img src='attachment/43. Full Architecture of Transformer.png' style='zoom:50%'>

1. Decoder中Masked Multi-Head Attention层，防止偷窥information in the future。

2. ②处的Multi-Head Attention，其中的$~Q~$来自Decoder，而$K,V$来自Encoder的输出。

# 3. ELMO

<table>
  <tr>
      <td><img src='attachment/43. ELMO.jpg' style='zoom:50%'></td>
      <td><img src='attachment/43. ELMO_Architecture.jpg' style='zoom:50%'></td>
    </tr>
</table>    


ELMO（Embeddings from Language Model）

RNN-based language models (trained from lots of sentences).

2018年3月份，ELMo出世，该paper是NAACL18 Best Paper。在之前2013年的word2vec及2014年的GloVe的工作中，每个词对应一个vector，对多义词无能为力。在ELMo世界里，预训练好的模型不再只是向量对应关系，而是一个训练好的模型。使用时，将一句话或一段话输入模型，<font color=blue>模型会根据上下文来推断每个词的词向量。</font>

<img src='attachment/43. ELMo_Embedding.png' style='zoom:50%'>

# 4. BERT

- General Architecture

- Masked Language Model

- Subword Units

- Pre-training and Fine-Tuning BERT

BERT (Bidirectional Encoder Representations from Transforers)

- BERT = <font color=red>Encoder of Transformer</font>

<table>
  <tr>
      <td><img src='attachment/43. BERT_Carton.jpg' style='zoom:50%'></td>
      <td><img src='attachment/43. Transformer_Encoder.jpg' style='zoom:50%'></td>
      <td><img src='attachment/43. BERT_GPT_ELMo.jpg' style='zoom:50%'></td>
    </tr>
</table>    

### 4.1 General Architecture

<img src='attachment/43. BERT_Embedding.jpg' style='zoom:50%'>

其中: 

- Token Embeddings是词向量，第一个单词是CLS标志，可以用于之后的分类任务；

- Segment Embeddings用来区别两种句子，因为预训练不关做LM，还要做以两个句子为输入的分类任务；

- Position Embeddings和Transformer不一样，不是三角函数而是学习出来的。

### 4.2 Masked Language Model

<img src='attachment/43. BERT_Masked_LM.jpg' style='zoom:50%'>

在fine-turning任务中，见不到\[MASK\]这个token，因此在pre-training阶段，mask是有一定技巧的:  

1. 随机mask 15%的词；

2. 对于被选中的词，80%的几率用\[MASK\]替换；10%几率被替换为其它词；10%几率不做处理。



- 如果只使用\[MASK\]替换，那么模型可能更多注意到被mask调的词，而would not have to learn good Representation for other words；

- 如果只使用\[MASK\]替换或随机替换成其它的token，那么模型可能会认为当前token总是错的；

- if we only use \[MASK\] token or observed word, model would be bias to trivially copy.

### 4.3 Next Sentence Prediction

<img src='attachment/43. BERT_Next_Sentence_Prediction.jpg' style='zoom:50%'>

### 4.4 Pre-training and Fine-truning

#### 1. Case1——句子分类

<img src='attachment/43. BERT_Case1.jpg' style='zoom:50%'>

#### 2. Case2——词性分类

<img src='attachment/43. BERT_Case2.jpg' style='zoom:50%'>

#### Case3——推理

<img src='attachment/43. BERT_Case3.jpg' style='zoom:50%'>

#### Case4——QA

<img src='attachment/43. BERT_Case4.png' style='zoom:50%'>